## Markov Chains

In [2]:
import numpy as np

In [6]:
transition_probabilities = [
        [0.7, 0.2, 0.0, 0.1],  # from s0 to s0, s1, s2, s3
        [0.0, 0.0, 0.9, 0.1],  # from s1 to ...
        [0.0, 1.0, 0.0, 0.0],  # from s2 to ...
        [0.0, 0.0, 0.0, 1.0],  # from s3 to ...
    ]

n_max_steps = 50

def print_sequence(start_state=0):
    current_state = start_state
    print("States:", end=" ")
    for step in range(n_max_steps):
        print(current_state, end=" ")
        if current_state == 3:
            break
        current_state = np.random.choice(range(4), p=transition_probabilities[current_state])
    else:
        print("...", end="")
    print()

for _ in range(10):
    print_sequence()

States: 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 0 0 3 
States: 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 0 0 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 ...
States: 0 0 0 0 0 3 
States: 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 
States: 0 0 0 0 0 3 
States: 0 3 
States: 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 ...
States: 0 0 0 0 0 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 2 1 3 


## Markov Decision Process

Markov decision processes are an extension of Markov chains;
the difference is the addition of actions (allowing choice) and rewards (giving motivation)

In [9]:
import numpy as np
nan=np.nan # represents impossible actions

#transition_probabilities
T = np.array([ # in s0, if action a0 then proba 0.7 to state s0 and 0.3 to state s1, etc.
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], [nan, nan, nan], [0.0, 0.0, 1.0]],
        [[nan, nan, nan], [0.8, 0.1, 0.1], [nan, nan, nan]],
    ])
# rewards
R = np.array([ 
        [[10., 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]],
        [[10., 0.0, 0.0], [nan, nan, nan], [0.0, 0.0, -50.]],
        [[nan, nan, nan], [40., 0.0, 0.0], [nan, nan, nan]],
    ])

possible_actions = [[0, 1, 2], [0, 2], [1]]

In [12]:
def policy_fire(state):
    return [0, 2, 1][state]

def policy_random(state):
    return np.random.choice(possible_actions[state])

def policy_safe(state):
    return [0, 0, 1][state]

class MDPEnvironment(object):
    def __init__(self, start_state=0):
        self.start_state=start_state
        self.reset()
    def reset(self):
        self.total_rewards = 0
        self.state = self.start_state
    def step(self, action):
        next_state = np.random.choice(range(3), p=T[self.state][action])
        reward = R[self.state][action][next_state]
        self.state = next_state
        self.total_rewards += reward
        return self.state, reward

def run_episode(policy, n_steps, start_state=0, display=True):
    env = MDPEnvironment()
    if display:
        print("States (+rewards):", end=" ")
    for step in range(n_steps):
        if display:
            if step == 10:
                print("...", end=" ")
            elif step < 10:
                print(env.state, end=" ")
        action = policy(env.state)
        state, reward = env.step(action)
        if display and step < 10:
            if reward:
                print("({})".format(reward), end=" ")
    if display:
        print("Total rewards =", env.total_rewards)
    return env.total_rewards

for policy in (policy_fire, policy_random, policy_safe):
    all_totals = []
    print(policy.__name__)
    for episode in range(1000):
        all_totals.append(run_episode(policy, n_steps=100, display=(episode<5)))
    print("Summary: mean={:.1f}, std={:1f}, min={}, max={}".format(np.mean(all_totals), np.std(all_totals), np.min(all_totals), np.max(all_totals)))
    print()

policy_fire
States (+rewards): 0 (10.0) 0 (10.0) 0 (10.0) 0 (10.0) 0 (10.0) 0 1 (-50.0) 2 (40.0) 0 1 (-50.0) ... Total rewards = 260.0
States (+rewards): 0 (10.0) 0 (10.0) 0 (10.0) 0 (10.0) 0 1 (-50.0) 2 (40.0) 0 1 (-50.0) 2 (40.0) ... Total rewards = 290.0
States (+rewards): 0 1 (-50.0) 2 (40.0) 0 1 (-50.0) 2 (40.0) 0 1 (-50.0) 2 (40.0) 0 ... Total rewards = 210.0
States (+rewards): 0 (10.0) 0 1 (-50.0) 2 2 (40.0) 0 (10.0) 0 (10.0) 0 1 (-50.0) 2 (40.0) ... Total rewards = 110.0
States (+rewards): 0 (10.0) 0 (10.0) 0 1 (-50.0) 2 2 (40.0) 0 (10.0) 0 (10.0) 0 (10.0) 0 ... Total rewards = 150.0
Summary: mean=127.6, std=129.863953, min=-390.0, max=520.0

policy_random
States (+rewards): 0 0 (10.0) 0 0 0 (10.0) 0 (10.0) 0 1 1 1 (-50.0) ... Total rewards = 160.0
States (+rewards): 0 (10.0) 0 (10.0) 0 0 (10.0) 0 0 (10.0) 0 0 1 (-50.0) 2 (40.0) ... Total rewards = -70.0
States (+rewards): 0 (10.0) 0 1 1 (-50.0) 2 (40.0) 0 0 1 1 1 (-50.0) ... Total rewards = 70.0
States (+rewards): 0 (10.0) 0 (

In [20]:
Q = np.full((3, 3), -np.inf) # -inf for impossible actions
Q[1, [0, 1]] = 0.0
print(Q)
Q = np.full((3, 3), -np.inf) # -inf for impossible actions
print(Q)

[[-inf -inf -inf]
 [  0.   0. -inf]
 [-inf -inf -inf]]
[[-inf -inf -inf]
 [-inf -inf -inf]
 [-inf -inf -inf]]


In [21]:
for state, actions in enumerate(possible_actions):
    print(state, actions)
    Q[state, actions] = 0.0 # Initial value = 0.0, for all possible actions
Q

0 [0, 1, 2]
1 [0, 2]
2 [1]


array([[  0.,   0.,   0.],
       [  0., -inf,   0.],
       [-inf,   0., -inf]])

In [27]:
learning_rate = 0.01
discount_rate = 0.8
n_iterations = 100
for iteration in range(n_iterations):
    Q_prev = Q.copy()
    for s in range(3): #for each state
        for a in possible_actions[s]: # for each action in state
            if iteration/99 == 1:
                print(s,a) 
            Q[s, a] = np.sum([T[s, a, sp] * (R[s, a, sp] + discount_rate * np.max(Q_prev[sp])) for sp in range(3)])

0 0
0 1
0 2
1 0
1 2
2 1


In [7]:
Q

array([[ 15.90909091,  12.72727273,  10.18181818],
       [  0.        ,         -inf, -13.3201581 ],
       [        -inf,  45.84980237,         -inf]])